In [2]:
## 1.1单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.1531],
        [-0.0987]], grad_fn=<AddmmBackward0>)

## 1.2 参数访问

In [3]:
# nn.Linear(8, 1)的weight以及bias
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2824,  0.2911,  0.0712, -0.2570, -0.1514, -0.0616, -0.0612, -0.1061]])), ('bias', tensor([-0.0370]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0370], requires_grad=True)
tensor([-0.0370])


In [5]:
net[2].weight.grad == None

True

## 1.3 一次性访问所有参数

In [6]:
print(*[(name, param.shape)for name,param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([-0.0370])

## 1.4 从嵌套块收集参数

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block_自定义 {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3974],
        [-0.3974]], grad_fn=<AddmmBackward0>)

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block_自定义 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_自定义 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_自定义 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_自定义 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 2.1参数初始化
### 2.1.1内置初始化

In [10]:
def init_normal(m):
    print(m)
    if type(m) == nn.Linear:
        print("XXX")
        nn.init.normal_(m.weight, mean=3, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0],net[2].weight.data[0], net[2].bias.data[0]

Linear(in_features=4, out_features=8, bias=True)
XXX
ReLU()
Linear(in_features=8, out_features=1, bias=True)
XXX
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


(tensor([2.9961, 2.9991, 3.0026, 3.0135]),
 tensor(0.),
 tensor([2.9929, 3.0004, 2.9940, 3.0087, 2.9834, 3.0119, 3.0024, 2.9921]),
 tensor(0.))

In [11]:
net_test = nn.Sequential(nn.Sequential(nn.Linear(4,1)),nn.Linear(4, 3), nn.ReLU(), nn.Linear(8, 2),nn.Sequential(nn.Linear(4,1)))
net_test.apply(init_normal)

Linear(in_features=4, out_features=1, bias=True)
XXX
Sequential(
  (0): Linear(in_features=4, out_features=1, bias=True)
)
Linear(in_features=4, out_features=3, bias=True)
XXX
ReLU()
Linear(in_features=8, out_features=2, bias=True)
XXX
Linear(in_features=4, out_features=1, bias=True)
XXX
Sequential(
  (0): Linear(in_features=4, out_features=1, bias=True)
)
Sequential(
  (0): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=True)
  )
  (1): Linear(in_features=4, out_features=3, bias=True)
  (2): ReLU()
  (3): Linear(in_features=8, out_features=2, bias=True)
  (4): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=True)
  )
)


Sequential(
  (0): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=True)
  )
  (1): Linear(in_features=4, out_features=3, bias=True)
  (2): ReLU()
  (3): Linear(in_features=8, out_features=2, bias=True)
  (4): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [19]:
net_ones = nn.Linear(4,1)
a = torch.rand(2,4)
net_ones(a),net_ones.weight,net_ones.bias

(tensor([[0.0675],
         [0.1068]], grad_fn=<AddmmBackward0>),
 Parameter containing:
 tensor([[-0.4484,  0.4221, -0.1396, -0.0455]], requires_grad=True),
 Parameter containing:
 tensor([0.1442], requires_grad=True))

## 2.2对不同的模块应用不同的初始化方法

In [20]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0207, -0.3487,  0.2831, -0.2391])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 2.3自定义初始化

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.4784, -6.2278,  5.6366,  9.5815],
        [ 0.0000,  5.7287,  0.0000, -8.8529]], grad_fn=<SliceBackward0>)

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.2278,  6.6366, 10.5815])

## 2.4参数绑定

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
